In [ ]:
import glob
import numpy as np
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
class ListOfStrings:
    def __init__(self, listOfStrings):
        self.listOfStrings = listOfStrings
        self.maxlen = np.max(list(map(len, listOfStrings)))
def gramize(listOfStrings, gramsize):
    X = listOfStrings.listOfStrings
    numele = len(X)
    reqdim = listOfStrings.maxlen - gramsize +1 
    Y = []
    for i in X:
        tempy = []
        paddedi = i + "0"*(listOfStrings.maxlen-len(i))
        for j in range(0,len(paddedi)-gramsize+1):
            tempy.append(paddedi[j:j+gramsize])
        if len(tempy) == 0:
            tempy = [paddedi]
        templeny = len(tempy)
        Y.append(tempy)
    return np.array(Y)
def ngramize(mingram, maxgram, listOfStrings):
    ngramData = np.array([])
    print("maxlen is")
    print(listOfStrings.maxlen)
    for gramsize in range(mingram, min(maxgram,listOfStrings.maxlen)+1):
        gramData = gramize(listOfStrings, gramsize)
        if ngramData.shape[0] == 0:
            ngramData = gramData.copy()
        else:
            ngramData = np.append(ngramData, gramData, axis=1)
    return ngramData
def removeHash(listOfStrings):
    return np.array([":".join(x.split(":")[1:]) for x in listOfStrings])
def getSentences(listOfStrings,mingram, maxgram, maxlen=None):
    listOfStringstemp = removeHash(listOfStrings)
    print(listOfStringstemp)
    listOfStringstemp = ListOfStrings(listOfStringstemp)
    if maxlen is not None:
        listOfStringstemp.maxlen = maxlen
    listOfListOfStringstemp = ngramize(mingram, maxgram, listOfStringstemp)
    print(listOfListOfStringstemp)
    listOfSentences = np.array([" ".join(x) for x in listOfListOfStringstemp])
    return listOfSentences, listOfStringstemp.maxlen
class BagOfWordsv1:
    def __init__(self, listOfStrings, mingram, maxgram):
        self.vectorizer = CountVectorizer()
        self.mingram = mingram
        self.maxgram = maxgram
        listOfSentences, self.maxlen = getSentences(listOfStrings, mingram, maxgram)
        baggedData = self.vectorizer.fit_transform(listOfSentences)
        self.tfidf = TfidfTransformer().fit(baggedData)
    def process(self, listOfStrings):
        listOfSentences, _ = getSentences(listOfStrings,self.mingram, self.maxgram, maxlen=self.maxlen)
        baggedData = self.vectorizer.transform(listOfSentences)
        baggedData = self.tfidf.transform(baggedData)
        return baggedData
def readFileByLines(filename):
    f = open(filename, 'r')
    lst = f.readlines()
    f.close()
#     print(lst)
    lst = list(map(str.rstrip, lst))
    return lst
    

In [ ]:
print("hello")
foldername = "metadata_maps/vdisk/*.db"
files = sorted(glob.glob(foldername))
ctr=0
alltemptableDatastr = []
ssTableDatastr = {}
for fle in files:
    thisline = readFileByLines(fle)
    # if(len(thisline)>1000):
    #     continue
    # thisline = [x.strip('\n') for x in thisline]

    ssTableDatastr[ctr] = np.array(thisline)
    ctr+=1
    alltemptableDatastr += thisline
    print(ctr,len(thisline))
# print(lines)
alltemptableDatastr = np.array(alltemptableDatastr)
print(ssTableDatastr)
print(alltemptableDatastr)
print("done")

In [ ]:
bagOfWords = BagOfWordsv1(alltemptableDatastr,6,6)

In [ ]:
ssTableDatastr0sparse = bagOfWords.process(ssTableDatastr[0])

In [ ]:
ssTableDatastr0sparse.toarray()

In [ ]:
ssTableDatastr0sparse.todense()

In [ ]:
ssTableDatastr0sparse

In [ ]:
len(ssTableDatastr[0])

In [ ]:
categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']


In [ ]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',
    categories=categories, shuffle=True, random_state=42)

In [ ]:
twenty_train.target_names

In [ ]:
len(twenty_train.data)

len(twenty_train.filenames)

In [ ]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))




print(twenty_train.target_names[twenty_train.target[0]])

In [ ]:
twenty_train.target[:10]

In [ ]:
for t in twenty_train.target[:10]:
    print(twenty_train.target_names[t])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

In [ ]:
count_vect.vocabulary_.get(u'algorithm')

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

In [ ]:
X_train_tf[0].todense().tolist()

In [ ]:
X_train_counts[0].todense().tolist()

In [ ]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

In [ ]:
X_train_tfidf[0].todense().tolist()

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [ ]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))
